In [29]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

In [30]:
def imprimeResult(results):
    media = results['test_score'].mean()
    print("Resultados cross validation:",results['test_score'])
    print("Média da precisão: {0:.2f}".format(media*100),'%')

    #Acurracy fica entre a média + 2(desvio_padrao) e a média - 2(desvio_padrao)
    desvio_padrao = results['test_score'].std()
    print("Desvio padrão: {0:.2f}".format(desvio_padrao*100),'%')

    acumulo_baixo = media - 2*desvio_padrao
    acumulo_alto = media + 2*desvio_padrao
    print("Acurracy esta entre: [",acumulo_baixo*100,"---", acumulo_alto*100,"]")

<h2>Criando uma nova coluna (aleatória)</h2>
<p>
   A coluna representa a possibilidade de em um futuro o modelo ter que prever com dados não treinados previamente 
</p>

In [31]:
import numpy as np

SEED = 123

np.random.seed(SEED)
np.random.randint(-2,2, size = 10000) #size é o numero de linhas do dataFrame

array([ 0, -1,  0, ..., -2, -1, -1])

In [32]:
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2,2, size = 10000)
#Deixa o menor valor da coluna possitivo
abss = abs(dados['modelo_aleatorio'].unique().min())
dados['modelo_aleatorio'] = dados['modelo_aleatorio'] + abss + 1    #somando 1 para que não seja 0
dados['modelo_aleatorio'].unique()

array([20, 21, 14,  3,  4, 18, 12, 16,  2, 19,  9, 13, 17, 15, 10,  6,  8,
       11,  7, 23, 22,  5,  1])

In [33]:
dados

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,20
1,40557.96,1,20,12622.05362,21
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,3
4,117384.68,1,4,12770.11290,4
...,...,...,...,...,...
9995,97112.86,0,12,25060.64248,14
9996,107424.63,1,16,21317.31764,19
9997,93856.99,0,4,20950.38812,7
9998,51250.57,1,7,16840.13376,8


In [34]:
#Aleatoriedade adicionada e, alguns valores aparecem mais que outros -> era isso que queriamos!
dados['modelo_aleatorio'].value_counts()

20    937
19    850
18    788
21    685
15    680
16    677
17    671
14    619
13    590
12    506
11    506
22    487
10    373
9     313
8     296
23    256
7     245
6     192
5     142
4      93
3      61
2      27
1       6
Name: modelo_aleatorio, dtype: int64

<h2>GroupKFold</h2>
<p>
    Usar o sufle ou o straify ou randomstate não irá treinar do modo que queremos.<br>
    Irá utilizar dados que estamos usando para verificar se o modelo prevê corretamente com 'dados futuros'.<br>
    Por isso iremos usar GroupKFold, ele irá tratar corretamento os 'dados futuros'.<br>
</p>

In [35]:
from sklearn.model_selection import GroupKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

cv = GroupKFold(n_splits = 5)

model = DecisionTreeClassifier(max_depth = 3)

result = cross_validate(model, x, y, cv = cv, groups = dados['modelo_aleatorio'])

imprimeResult(result)

Resultados cross validation: [0.78496241 0.7967968  0.77138569 0.7827381  0.79618474]
Média da precisão: 78.64 %
Desvio padrão: 0.94 %
Acurracy esta entre: [ 76.75511129385336 --- 80.5275979002337 ]


<h4>A coluna 'modelo_aleatorio' não é usada para treino, mas é para o teste</h4>